In [ ]:
import requests
from time import sleep

In [ ]:
url = 'URL-OF-API'
headers = {'Api-Token': "XXXXXXXXXXXXXXXXXXXXX"}

# Create an Entry to the Catalog

In [ ]:
data = {"package_metadata": {"title": "Test Workflow for UC A3",
                             "notes": "This workflow performs entity extraction and linking",
                             "tags": ["STELAR", "Entity extraction", "Entity linking"]}}
response = requests.post(url + 'workflow/publish', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())
package_id = response.json()['result']['package_id']

# Create a workflow execution

In [ ]:
data = {"tags": {}}
response = requests.post(url + 'workflow/execution/create', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())
workflow_exec_id = response.json()['workflow_exec_id']

# Create a task for Entity Extraction

In [ ]:
data = {"workflow_exec_id": workflow_exec_id,
        "docker_image": "ner",
        "input": [  "42d9de05-530d-4c87-b2a1-f4a661d637f5"],
        "parameters": {
             "output_file":"test_output",
             "text_column":"description",
             "product_column":"product",
             "csv_delimiter":",",
             "keep_food": True,
             "N":100,
             "extraction_type":["food"],
             "model":["instafoodroberta"],
             "syntactic_analysis_tool":"spacy"
        },
        "package_id": package_id,
        "tags": {}}
response = requests.post(url + 'task/execution/create', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())
task_exec_id = response.json()['task_exec_id']

# Track task metadata

In [ ]:
state = 'running'
while state != 'succeeded' and state != 'failed':
    response = requests.get(url + 'task/execution/read?id=' + task_exec_id, 
                        headers=headers)
    print(response.status_code)
    print(response.json())   
    state = response.json()['result']['metadata']['state']
    sleep(10)

In [ ]:
resource_id = response.json()['result']['output'][0]

# Create a task for Entity Linking

In [ ]:
data = {"workflow_exec_id": workflow_exec_id,
        "docker_image": "alzeakis/pytokenjoin:v4",
        "input": [  resource_id,
                   'dbee0078-f053-404a-b09d-bde29eaf5521'],
        "parameters": {
            "col_id_left": 1,
            "col_text_left": 2,
            "col_separator_left": ",",
            "text_separator_left": " ",
            "col_id_right": 1,
            "col_text_right": 2,
            "col_separator_right": ";",
            "text_separator_right": " ",            
            "k": 1,
            "delta_alg": "1",
            "output_file": "out.csv",
            "method": "knn",
            "similarity": "jaccard",
            "foreign": "foreign"
        },
        "package_id": package_id,
        "tags": {}}
response = requests.post(url + 'task/execution/create', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())
task_exec_id = response.json()['task_exec_id']

# Track task metadata

In [ ]:
state = 'running'
while state != 'succeeded' and state != 'failed':
    response = requests.get(url + 'task/execution/read?id=' + task_exec_id, 
                        headers=headers)
    print(response.status_code)
    print(response.json())   
    state = response.json()['result']['metadata']['state']
    sleep(2)

# Commit workflow metadata

In [ ]:
data = {"workflow_exec_id": workflow_exec_id, "state": "succeeded"}
response = requests.post(url + 'workflow/execution/commit', 
                          json=data, headers=headers)
print(response.status_code)
print(response.json())  

# Get workflow metadata

In [ ]:
response = requests.get(url + 'workflow/execution/read?id=' + workflow_exec_id, 
                        headers=headers)
print(response.status_code)
print(response.json())  